# Prueba Técnica

### Laura Camila Rincón Manrique

In [154]:
#pip install requests
#!pip install json

In [201]:
import requests
import json
from datetime import datetime, timedelta
import os

import pandas as pd

In [202]:
def Get_data_dates(url,date):
    url = f"{url}{date}"
    api_response = requests.get(url)
    if api_response.status_code == 200:
        return api_response.json()
    else:
        print(f"Error al obtener los datos para la fecha {date}, Error API: {api_response.status_code}")
        return None

In [203]:
def Verify_file_exist(path_file):   
    if os.path.exists(path_file):
        os.remove(path_file)
        print(f"The file {path_file} was delete.")
    else:
        print(f"The file {path_file} doesn't exist and It is goint to create.")

In [204]:
def Save_json(data,path_file):
    with open(path_file, 'w') as archivo:
        json.dump(data, archivo, indent=4)
    print(f"Datos almacenados en {path_file}")

In [207]:
#Data Range
initial_date = datetime(2024, 1, 1)
end_date = datetime(2024, 1, 31) 
actual_date = initial_date
url = "http://api.tvmaze.com/schedule/web?date="
archivo = 'ejemplo.json'


list_data = []

while actual_date <= end_date:
    date_str = actual_date.strftime('%Y-%m-%d')  #YYYY-MM-DD
    path_file = f'C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_{date_str}.json'
    print(f"Getting the data to: {date_str}")

    data = Get_data_dates(url,date_str)

    if data:
        list_data.extend(data)
        Verify_file_exist(path_file)
        Save_json(data,path_file)

    actual_date += timedelta(days=1)



Getting the data to: 2024-01-01
The file C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-01.json was delete.
Datos almacenados en C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-01.json
Getting the data to: 2024-01-02
The file C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-02.json was delete.
Datos almacenados en C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-02.json
Getting the data to: 2024-01-03
The file C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-03.json was delete.
Datos almacenados en C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-03.json
Getting the data to: 2024-01-04
The file C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-04.json was delete.
Datos almacenados en C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-04.json
Getting the data to: 2024-01-05
The file C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-05.json was delete.
Datos almacenados en C:/Users/

## Convert in dataframe

In [208]:
episodes_df = pd.DataFrame(columns=["id_index","id","url","name","season","number","type","airdate","airtime",
                                                 "airstamp","runtime","rating_average","image","summary"])
links_episodes_df = pd.DataFrame(columns=["id_index","id_episodes","self_href","show_href","show_name"])
embedded_df = pd.DataFrame(columns=["id_index","id_episodes","id","url","name","type","language","status","runtime",
                                    "averageRuntime","premiered","ended","officialSite","schedule_time","rating_average","weight","network","dvdCountry","summary","updated"])
genres_df = pd.DataFrame(columns=["id_index","id_episodes","genres"])
days_df = pd.DataFrame(columns=["id_index","id_episodes","days"])
webChannel_df = pd.DataFrame(columns=["id_index","id_episodes","id","name","country_name","country_code",
                                      "country_timezone","officialSite"])
image_df = pd.DataFrame(columns=["id_index","id_episodes","medium","original"])
externals_df = pd.DataFrame(columns=["id_index","id_episodes","tvrage","thetvdb","imdb"])
links_show_df = pd.DataFrame(columns=["id_index","id_episodes","self_href","previousepisode_href","previousepisode_name"])

In [209]:
genres_df.head()

,id_index,id_episodes,genres


In [210]:
#Create the links dataframe
def Links_dataframe(dic_data,id,row):
    links_episodes_df.loc[row,'id_episodes'] = id
    links_episodes_df.loc[row,'id_index'] = row + 1
    links_episodes_df.loc[row,'self_href'] = dic_data.get('self',{}).get('href')
    links_episodes_df.loc[row,'show_href'] = dic_data.get('show',{}).get('href')
    links_episodes_df.loc[row,'show_name'] = dic_data.get('show',{}).get('name')


In [211]:
#Create the links_show dataframe
def Links_show_dataframe(dic_data,id,row):
    links_show_df.loc[row,'id_episodes'] = id
    links_show_df.loc[row,'id_index'] = row + 1
    links_show_df.loc[row,'self_href'] = dic_data.get('self',{}).get('href')
    links_show_df.loc[row,'previousepisode_href'] = dic_data.get('previousepisode',{}).get('href')
    links_show_df.loc[row,'previousepisode_name'] = dic_data.get('previousepisode',{}).get('name')

In [212]:
#Convert the simple dict to dataframe Use in image and externals
def dict_to_dataframe(dic_data,id,df):
    index_df = len(df)
    for k in dic_data.keys():
        df.loc[index_df,'id_index'] = index_df + 1
        df.loc[index_df,'id_episodes'] = id
        df.loc[index_df,k] = dic_data.get(k)
        #index_df += 1


In [213]:
#convert the simple list to dataframe
def list_to_dataframe(list_data,df,column_name,id):
    index_df = len(df)
    for i in list_data:
        df.loc[index_df,'id_index'] = index_df + 1
        df.loc[index_df,'id_episodes'] = id
        df.loc[index_df,column_name] = i
        index_df += 1

In [214]:
#Create the genres dataframe
def Genres_dataframe(list_data,id):
    index_df = len(genres_df)
    for i in list_data:
        genres_df.loc[index_df,'id_index'] = index_df + 1
        genres_df.loc[index_df,'id_episodes'] = id
        genres_df.loc[index_df,'genres'] = i
        index_df += 1

In [215]:
#Create the days dataframe
def Days_schedule_dataframe(list_data,id):
    index_df = len(genres_df)
    for i in list_data:
        days_df.loc[index_df,'id_index'] = index_df + 1
        days_df.loc[index_df,'id_episodes'] = id
        days_df.loc[index_df,'days'] = i
        index_df += 1

In [216]:
#Create the WebChannel dataframe
def WebChannel_dataframe(dic_data,id,row):
    dic_keys = dic_data.keys()
    webChannel_df.loc[row,'id_episodes'] = id
    webChannel_df.loc[row,'id_index'] = row + 1
    for k in dic_keys:
        #print(k)
        #print(isinstance(dic_data.get(k), dict))
        if not(isinstance(dic_data.get(k), dict)):
            #print(dic_data.get(k))
            webChannel_df.loc[row,k] = dic_data.get(k)
        elif k == "country":
            webChannel_df.loc[row,'country_name'] = dic_data.get(k,{}).get('name')
            webChannel_df.loc[row,'country_code'] = dic_data.get(k,{}).get('code')
            webChannel_df.loc[row,'country_timezone'] = dic_data.get(k,{}).get('timezone')

In [217]:
#Create the Embedded dataframe
def Embedded_dataframe(dic_data,id,row):
    dic_keys = dic_data.keys()
    embedded_df.loc[row,'id_episodes'] = id
    embedded_df.loc[row,'id_index'] = row + 1
    for k in dic_keys:
        #print(k)
        #print(isinstance(dic_data.get(k), dict))
        if not(isinstance(dic_data.get(k), dict)) and not(isinstance(dic_data.get(k), list)):
            #print(dic_data.get(k))
            embedded_df.loc[row,k] = dic_data.get(k)
        elif k == "genres":
            list_to_dataframe(dic_data.get(k),genres_df,'genres',id)
            #Genres_dataframe(dic_data.get(k),id)
        elif k == "schedule":
            embedded_df.loc[row,'schedule_time'] = dic_data.get(k,{}).get('time')
            list_to_dataframe(dic_data.get(k,{}).get('days'),days_df,'days',id)
            #Days_schedule_dataframe(dic_data.get(k,{}).get('days'),id)
        elif k == "rating":
            embedded_df.loc[row,'rating_average'] = dic_data.get(k,{}).get('average')
        elif k == "webChannel":
            WebChannel_dataframe(dic_data.get(k,),id,row)
        elif k == "externals":
            dict_to_dataframe(dic_data.get(k,),id,externals_df)
        elif k == "image":
            dict_to_dataframe(dic_data.get(k,),id,image_df)
        elif k == "_links":
            Links_show_dataframe(dic_data.get(k,),id,row)
            
        


In [218]:
def Create_dataframe(dic_data,row):
    dic_keys = dic_data.keys()
    episodes_df.loc[row,"id_index"] = row + 1
    for k in dic_keys:
        #print(k)
        #print(isinstance(dic_data.get(k), dict))
        if not(isinstance(dic_data.get(k), dict)):
            #print(dic_data.get(k))
            episodes_df.loc[row,k] = dic_data.get(k)
        elif k == "rating":
            episodes_df.loc[row,'rating_average'] = dic_data.get(k,{}).get('average')
        elif k == "_links":
            Links_dataframe(dic_data.get(k),dic_data.get('id'),row)
        elif k == "_embedded":
            Embedded_dataframe(dic_data.get(k,{}).get("show",{}),dic_data.get('id'),row)
            
    

In [219]:
Create_dataframe(list_data[0],0)

In [220]:
links_show_df.head()

,id_index,id_episodes,self_href,previousepisode_href,previousepisode_name
0,1,2730586,https://api.tvmaze.com/shows/51908,https://api.tvmaze.com/episodes/2730595,Серия 10
